# New guess function

In [1]:
# imports
import enum
import random
import numpy as np
import math
from numpy import pi
from qiskit import Aer, QuantumRegister, ClassicalRegister, QuantumCircuit, execute

In [2]:
class GuessStrategy(enum.Enum):
    one_bit_same_as_measured = 1
    two_bit_base = 2
    two_bit_neural_network = 3

In [3]:
def guess_lambda_used_one_bit_strategy(counts: str) -> int:
    """ Decides which lambda was used on the real execution from the one bit 'counts' measured
        It is a silly guess.
        It returns the same lambda index used as the measured result
    """
    if len(counts) != 1:
        raise ValueError('counts MUST be a one character length string')
    if "0" in counts:
        return 0
    return 1

In [4]:
def guess_lambda_used_two_bit_strategy(counts: str) -> int:
    """ Decides which lambda was used on the real execution from the two 'counts' measured
        Setting eta0 >= eta1:
            * outcome 00 -> eta1 as the most probable (more attenuation)
            * outcome 01 -> eta0 as the most probable (less attenuation) 
            * outcome 10 -> 50% chance, random choice
            * outcome 11 -> not possible, but in case we get it (from noisy simulation), 50% chance, random choice
    """
    if len(counts) != 2:
        raise ValueError('counts MUST be a two character length string')
    if counts == "00":
        return 1
    if counts == "01":
        return 0
    if counts == "10" or counts == "11":
        return random.choice([0,1])
    

In [5]:
def get_measured_value_from_counts(counts_dict: dict) -> str:
    """ converts the dictionary counts measured to the
        value measured in string format
    """
    if len(list(counts_dict.keys())) != 1:
        raise ValueError('Circuit execution shots MUST be 1')
    return list(counts_dict.keys())[0]

In [6]:
def convert_counts_to_lambda_used(counts_dict: dict, guess_strategy: GuessStrategy = GuessStrategy.one_bit_same_as_measured) -> int:
    """ Decides which lambda was used on the real execution from the 'counts' measured
        based on the guess strategy that is required to use
    """
    if (guess_strategy != GuessStrategy.one_bit_same_as_measured and 
        guess_strategy != GuessStrategy.two_bit_base and 
        guess_strategy != GuessStrategy.two_bit_neural_network):
            raise ValueError('Invalid Guess Strategy')

    counts = get_measured_value_from_counts(counts_dict)
    
    if guess_strategy == GuessStrategy.one_bit_same_as_measured:
        return guess_lambda_used_one_bit_strategy(counts)
    if guess_strategy == GuessStrategy.two_bit_base:
        return guess_lambda_used_two_bit_strategy(counts)
    if guess_strategy == GuessStrategy.two_bit_neural_network:
        raise NotImplementedError('Guess Strategy not implemented yet')

In [7]:
def compute_damping_channel_measuring_one_qubit() -> int:
    theta = pi/4
    attenuation_factor = 0.4
    
    backend = backend = Aer.get_backend('qasm_simulator')
    qreg_q = QuantumRegister(2, 'q')
    creg_c = ClassicalRegister(1, 'c')

    initial_state = (math.cos(theta), math.sin(theta))

    circuit = QuantumCircuit(qreg_q, creg_c)
    circuit.initialize([initial_state[0],
                        initial_state[1]], qreg_q[0])
    circuit.reset(qreg_q[1])
    circuit.cry(2 * np.arcsin(np.sqrt(attenuation_factor)), qreg_q[0], qreg_q[1])
    circuit.cx(qreg_q[1], qreg_q[0])
    circuit.rx(0, qreg_q[0])
    circuit.ry(0, qreg_q[0])
    circuit.measure(qreg_q[0], creg_c[0])

    counts = execute(circuit, backend, shots=1).result().get_counts(circuit)
    print(f'counts returned: {counts}')
    return convert_counts_to_lambda_used(counts, guess_strategy=GuessStrategy.one_bit_same_as_measured)

In [8]:
def compute_damping_channel_measuring_two_qubits() -> int:
    theta = pi/4
    attenuation_factor = 0.4
    
    backend = backend = Aer.get_backend('qasm_simulator')
    qreg_q = QuantumRegister(3, 'q')
    creg_c = ClassicalRegister(2, 'c')

    initial_state = (0, math.cos(theta), math.sin(theta), 0)

    circuit = QuantumCircuit(qreg_q, creg_c)
    circuit.initialize(initial_state, [0, 1])
    circuit.reset(qreg_q[2])
    circuit.cry(2 * np.arcsin(np.sqrt(attenuation_factor)), qreg_q[1], qreg_q[2])
    circuit.cx(qreg_q[2], qreg_q[1])
    circuit.rx(1, qreg_q[1])
    circuit.ry(1, qreg_q[1])
    circuit.measure([0, 1], creg_c)

    counts = execute(circuit, backend, shots=1).result().get_counts(circuit)
    print(f'counts returned: {counts}')
    return convert_counts_to_lambda_used(counts, guess_strategy=GuessStrategy.two_bit_base)

In [9]:
compute_damping_channel_measuring_one_qubit()

counts returned: {'0': 1}


0

In [10]:
compute_damping_channel_measuring_two_qubits()

counts returned: {'00': 1}


1